### Project ini adalah Text Preprocessing untuk mengambil model tf-idf
### Sekaligus Prototipe untuk Melakukan Sistem Rekomendasi User To Item

In [1]:
import pandas as pd
df_wisata = pd.read_csv("../0. Dataset/Dataset_Toba_Text_Processed.csv")
df_history = pd.read_csv("../0. Dataset/userHistory.csv")

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(df_wisata['tags_joined'])


In [3]:
print(tfidf)

TfidfVectorizer()


In [4]:
import pickle
from scipy.sparse import save_npz

# with open("./Save Model/tfidf_vectorizer.pkl", "wb") as f:
#     pickle.dump(tfidf, f)

save_npz("./Save Model/tfidf_matrix.npz", tfidf_matrix)


In [5]:
user_id = 1
user_history = df_history[df_history['userID'] == user_id]['namaWisata'].tolist()
# df_wisata.head()
df_history.head()

,userID,namaWisata
0,1,Hill of Gibeon
1,1,Bukit Indah Sitalmak Talmak Sihotang
2,1,Bukit Senyum
3,1,Bukit Beta Tuk-tuk
4,2,Pantai Batuhoda


In [6]:
visited_wisata = df_wisata[df_wisata['title'].isin(user_history)]
visited_wisata.head(1)

,title,link,image_url,rating,reviews,address,opening_hours,latitude,longitude,kategori,aktivitas,deskripsi,kecamatan,biaya_masuk,biaya_parkir_motor,biaya_parkir_mobil,tags_joined
0,Hill of Gibeon,https://www.google.com/maps/place/Hill+of+Gibe...,https://drive.google.com/drive/folders/1AVbEcO...,4.5,457,"Kabupaten Toba Samosir, 21174, Sionggang Utara...",Open 24 hours,2.590898,98.9978849865071,Alam,"Aktivitas Air, Berenang, Pemandangan, Santai, ...","Terletak di Kabupaten Toba, Sumatera Utara, Bu...",Lumban Julu,10000,2000,5000,aktivitas air berenang pemandangan santai foto...


In [7]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
visited_indices = df_wisata[df_wisata['title'].isin(user_history)].index
user_profile_matrix = tfidf_matrix[visited_indices].mean(axis=0)
user_profile = np.asarray(user_profile_matrix).reshape(1, -1)

In [8]:
print(user_profile.shape)

(1, 921)


In [9]:
from sklearn.metrics.pairwise import cosine_similarity

similarities = cosine_similarity(user_profile, tfidf_matrix)
#print(similarities)

In [10]:

similarities = similarities.flatten()
#ngubah array multidimensi jadi 1 array
df_wisata['similarity'] = similarities
df_wisata.head(1)



,title,link,image_url,rating,reviews,address,opening_hours,latitude,longitude,kategori,aktivitas,deskripsi,kecamatan,biaya_masuk,biaya_parkir_motor,biaya_parkir_mobil,tags_joined,similarity
0,Hill of Gibeon,https://www.google.com/maps/place/Hill+of+Gibe...,https://drive.google.com/drive/folders/1AVbEcO...,4.5,457,"Kabupaten Toba Samosir, 21174, Sionggang Utara...",Open 24 hours,2.590898,98.9978849865071,Alam,"Aktivitas Air, Berenang, Pemandangan, Santai, ...","Terletak di Kabupaten Toba, Sumatera Utara, Bu...",Lumban Julu,10000,2000,5000,aktivitas air berenang pemandangan santai foto...,0.617864


In [11]:
# Hapus yang sudah dikunjungi
rekomendasi = df_wisata[~df_wisata['title'].isin(user_history)]
rekomendasi = rekomendasi.sort_values(by='similarity', ascending=False)

# Tampilkan Top-N
top_n = 5
print(rekomendasi[['title', 'similarity']].head(top_n))

                          title  similarity
98           Bukit Cinta Harian    0.488195
123            Bukit Sipatungan    0.484632
60   Bukit Cinta Silahisabungan    0.420330
18        Bukit Holbung Samosir    0.409342
113                Bukit Pahoda    0.392595
